## Environment Setup

In [1]:
!pip install openai
!pip install datasets
!pip install evaluate

In [2]:
import json
from openai import OpenAI
from IPython.utils.capture import capture_output
from datasets import load_dataset
import pprint

In [3]:
from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/MyDrive/CS 159 Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1LeLbrNZayyv5i4FVOL44lq8EKgwzSuZl/CS 159 Project


In [4]:
from factsheet_utils import *
from metrics_utils import *
from summary_utils import *

User token is required.

In [ ]:
client = OpenAI(api_key = input())

## Input Source Text

In [10]:
raw_case = input('Paste raw case file here: ')

Paste raw case file here: IN THE SUPREME COURT OF CALIFORNIA THE PEOPLE, Plaintiff and Respondent, v. FRANCISCO BURGOS et al., Defendants and Appellants. S274743 Sixth Appellate District H045212 Santa Clara County Superior Court C1518795, C1756994 June 3, 2024 Chief Justice Guerrero authored the opinion of the Court, in which Justices Corrigan, Kruger, Groban, and Jenkins concurred. Justice Groban filed a concurring opinion, in which Justice Corrigan concurred. Justice Evans filed a dissenting opinion, in which Justice Liu concurred. 1 PEOPLE v. BURGOS S274743 Opinion of the Court by Guerrero, C. J. In 2021, the Legislature passed Assembly Bill No. 333 (2021–2022 Reg. Sess.) (Assembly Bill 333), known as the STEP Forward Act of 2021. (Stats. 2021, ch. 699, § 1.) Assembly Bill 333 amended Penal Code section 186.22 by imposing new substantive requirements relating to gang enhancements and the criminal offense of gang participation. 1 (Stats. 2021, ch. 699, § 4.) Assembly Bill 333 also ad

In [7]:
pprint.pprint(raw_case)

('IN THE SUPREME COURT OF CALIFORNIA THE PEOPLE, Plaintiff and Respondent, v. '
 'FRANCISCO BURGOS et al., Defendants and Appellants. S274743 Sixth Appellate '
 'District H045212 Santa Clara County Superior Court C1518795, C1756994 June '
 '3, 2024 Chief Justice Guerrero authored the opinion of the Court, in which '
 'Justices Corrigan, Kruger, Groban, and Jenkins concurred. Justice Groban '
 'filed a concurring opinion, in which Justice Corrigan concurred. Justice '
 'Evans filed a dissenting opinion, in which Justice Liu concurred. 1 PEOPLE '
 'v. BURGOS S274743 Opinion of the Court by Guerrero, C. J. In 2021, the '
 'Legislature passed Assembly Bill No. 333 (2021–2022 Reg. Sess.) (Assembly '
 'Bill 333), known as the STEP Forward Act of 2021. (Stats. 2021, ch. 699, § '
 '1.) Assembly Bill 333 amended Penal Code section 186.22 by imposing new '
 'substantive requirements relating to gang enhancements and the criminal '
 'offense of gang participation. 1 (Stats. 2021, ch. 699, § 4.) A

## Generate Factsheets

In [8]:
# Break source up into segments for processing
print('Generating fact sheet for each document in source...')
source = break_text(raw_case)
source_responses = []
for s in source:
  source_responses.append(generate_fact_sheet(s, client, verbose=False))

# combine source responses
print('Combining source fact sheets...')
source_sheet = combine_fact_sheets(source_responses, client, verbose=False)
print(source_sheet)

Generating fact sheet for each document in source...
Combining source fact sheets...
```json
{
    "Case Information": "IN THE SUPREME COURT OF CALIFORNIA THE PEOPLE, Plaintiff and Respondent, v. FRANCISCO BURGOS et al., Defendants and Appellants. S274743 Sixth Appellate District H045212 Santa Clara County Superior Court C1518795, C1756994 June 3, 2024 Chief Justice Guerrero authored the opinion of the Court, in which Justices Corrigan, Kruger, Groban, and Jenkins concurred. Justice Groban filed a concurring opinion, in which Justice Corrigan concurred. Justice Evans filed a dissenting opinion, in which Justice Liu concurred.",
    "Parties Involved": "The People, Francisco Burgos, Damon Stevenson Jr., James Richardson, Derrik Lozano, Gregory Byrd, Defendant Francisco Burgos, Defendant Damon Stevenson Jr., Defendant James Richardson, Plaintiff and Respondent (represented by the Attorney General)",
    "Issue": "Whether the bifurcation provisions of section 1109, enacted by Assembly Bil

## Generate Summary

We can generate summaries now with Chain of Density Prompting. The user can adjust parameters such as approximate length of summary output (using the `num_words` parameter) and desired audience (using the `initial_prompt` parameter).

In [9]:
num_words = input('Number of words (Tiny - 25, Short - 130, Long - 650): ')
inital_prompt = "Summarize the factsheet so that it is understandable to the general public. Refer to the case by official name (partyX v. partyY)."
responses = chain_of_density_prompting(source_sheet, inital_prompt, client, ground_truth=None, num_words=int(num_words))
pprint.pprint(responses[-1][0])

Number of words (Tiny - 25, Short - 130, Long - 650): 100
('In The People v. Francisco Burgos et al., the Supreme Court of California '
 "ruled that Penal Code section 1109's bifurcation provisions do not apply "
 'retroactively to cases with gang enhancements. The decision emphasized '
 'legislative intent, focusing on preventing unfair prejudice and wrongful '
 'convictions. Chief Justice Guerrero authored the opinion with Justices '
 'Corrigan, Kruger, Groban, and Jenkins concurring. Justice Groban filed a '
 'concurring opinion, while Justice Evans dissented with Justice Liu. The '
 'Court reversed the judgment, stressing the importance of clarity in '
 'legislative intent on retroactivity. This ruling affects similar pending '
 'cases and illustrates the complexity of applying statutory provisions '
 'retrospectively for fair legal outcomes.')
